## Note: this doesn't work super reliably yet, mainly because the CAN messages for pos/vel gains aren't updated in ODrive firmware.

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport communicate
from communicate import CableRobot, MotorState, ControllerState
import numpy as np
import time
from IPython.display import clear_output

In [ ]:
PORT = '/dev/tty.usbmodem103568503'
start_positions = {} # axis -> position

In [ ]:
def configure_anticogging(axis, robot=None):
    with CableRobot(print_raw=True, silent=False, write_timeout=None, port=PORT) as robot:
        if hasattr(axis, '__iter__'):
            axes = axis
            del axis
        else:
            axes = [axis]
        for axis in axes:
            robot.update()
            robot.send(f'c{axis}')  # calibrate
        robot.silent = True
        print(robot.log_data['controller'])
        print('Waiting to begin calibration...')
        for axis in axes:
            while robot.log_data['motors'][axis] is None or robot.log_data['motors'][axis].state <= 1: # wait for calibration to begin
                robot.update()
                time.sleep(0.05)
        print('Waiting to finish calibration...')
        for axis in axes:
            while robot.log_data['motors'][axis].state != 1: # wait for calibration to finish
                robot.update()
                time.sleep(0.05)
        print('Starting anticogging calibration!')
        time.sleep(0.2)
        robot.update()
        for axis in axes:
            start_positions[axis] = robot.log_data['motors'][axis].length
        robot.silent = False
        for axis in axes:
            robot.send(f'{axis}n27c0.2,5.0')  # set vel gains
            robot.send(f'{axis}n26c50.0')  # set pos gain
            robot.send(f'{axis}n11c3,1')  # set controller mode to position, passthrough
            robot.send(f'{axis}n7c8')  # start closed loop control
            robot.send(f'{axis}n16c')  # start anticogging
        robot.update()
        time.sleep(0.5)
        robot.update()
def set_gains(axis, vel_gain, vel_integrator_gain, pos_gain):
    with CableRobot(print_raw=True, silent=False, write_timeout=None, port=PORT) as robot:
        robot.send(f'{axis}n26c{pos_gain}')  # set pos gain
        robot.send(f'{axis}n27c{vel_gain},{vel_integrator_gain}')  # set vel gains
        robot.update()
        time.sleep(0.1)
        robot.update()
def enable_all():
    with CableRobot(print_raw=True, silent=False, write_timeout=None, port=PORT) as robot:
        for i in range(4):
            robot.send(f'{i}n7c8')  # start closed loop control
        robot.update()

In [ ]:
configure_anticogging([0, 1, 2, 3])

In [ ]:
enable_all()

In [ ]:
for i in range(4):
    set_gains(i, 0.5, 5.0, 30.0)

In [ ]:
def monitor_anticogging(axes=[0, 1, 2, 3]):
    with CableRobot(print_raw=True, silent=True, write_timeout=None, port=PORT) as robot:
        while True:
            clear_output(wait=True)
            robot.update()
            if robot.log_data['motors'][axes[0]] is None:
                continue
            ls = [robot.log_data['motors'][axis].length - start_positions[axis] for axis in axes]
            print('  '.join(f'{l:7.4f}' for l in ls))
            time.sleep(0.03)
try:
    monitor_anticogging([0, 1, 2, 3])
except KeyboardInterrupt:
    pass